# Create schema and tables for Fabric Mirroring demo
This notebook connects to Azure SQL using SQL Authentication and creates:
- Schema: `dbo` (default)
- Tables: `DimCountry`, `DimStore`, `DimProduct`, `FactSales`

In [2]:
# Load configuration from .env file
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
SQL_SERVER_NAME = os.getenv('SQL_SERVER_NAME')
SQL_DATABASE = os.getenv('SQL_DATABASE', 'fabricMirrorDemoDb')
SQL_USER = os.getenv('SQL_USER')
SQL_PASSWORD = os.getenv('SQL_PASSWORD')

# Validate required configuration
if not all([SQL_SERVER_NAME, SQL_USER, SQL_PASSWORD]):
    raise ValueError("Missing required environment variables. Please check your .env file.")
    
# Build full server name
SQL_SERVER = f"{SQL_SERVER_NAME}.database.windows.net"
print(f"Configuration loaded for server: {SQL_SERVER}, database: {SQL_DATABASE}")

Configuration loaded for server: sql-mirror-westus3-flopes.database.windows.net, database: fabricMirrorDemoDb


In [3]:
import pyodbc
import textwrap

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={SQL_SERVER};"
    f"DATABASE={SQL_DATABASE};"
    f"UID={SQL_USER};"
    f"PWD={SQL_PASSWORD};"
    "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
)
conn_str = ''.join(conn_str)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print('Connected to Azure SQL')

Connected to Azure SQL


In [4]:
# Create tables with Product dimension and enhanced timestamps
ddl = textwrap.dedent('''
IF NOT EXISTS (SELECT 1 FROM sys.schemas WHERE name = 'dbo') EXEC('CREATE SCHEMA dbo');
;
IF OBJECT_ID('dbo.DimCountry','U') IS NULL BEGIN
  CREATE TABLE dbo.DimCountry (
    CountryKey   INT IDENTITY(1,1) PRIMARY KEY,
    CountryCode  NVARCHAR(3) NOT NULL,
    CountryName  NVARCHAR(100) NOT NULL,
    Iso3         NVARCHAR(3) NULL,
    CreatedAt    DATETIME2(3) NOT NULL DEFAULT SYSUTCDATETIME()
  );
  CREATE UNIQUE INDEX IX_DimCountry_CountryCode ON dbo.DimCountry(CountryCode);
END;
;
IF OBJECT_ID('dbo.DimProduct','U') IS NULL BEGIN
  CREATE TABLE dbo.DimProduct (
    ProductKey   INT IDENTITY(1,1) PRIMARY KEY,
    ProductCode  NVARCHAR(20) NOT NULL,
    ProductName  NVARCHAR(200) NOT NULL,
    Category     NVARCHAR(100) NULL,
    UnitPrice    DECIMAL(18,2) NULL,
    CreatedAt    DATETIME2(3) NOT NULL DEFAULT SYSUTCDATETIME()
  );
  CREATE UNIQUE INDEX IX_DimProduct_ProductCode ON dbo.DimProduct(ProductCode);
END;
;
IF OBJECT_ID('dbo.DimStore','U') IS NULL BEGIN
  CREATE TABLE dbo.DimStore (
    StoreKey     INT IDENTITY(1,1) PRIMARY KEY,
    StoreCode    NVARCHAR(20) NOT NULL,
    StoreName    NVARCHAR(200) NOT NULL,
    CountryKey   INT NOT NULL,
    OpenDate     DATE NULL,
    CreatedAt    DATETIME2(3) NOT NULL DEFAULT SYSUTCDATETIME(),
    CONSTRAINT FK_DimStore_DimCountry FOREIGN KEY (CountryKey) REFERENCES dbo.DimCountry(CountryKey)
  );
  CREATE UNIQUE INDEX IX_DimStore_StoreCode ON dbo.DimStore(StoreCode);
END;
;
IF OBJECT_ID('dbo.FactSales','U') IS NULL BEGIN
  CREATE TABLE dbo.FactSales (
    SalesId      BIGINT IDENTITY(1,1) PRIMARY KEY,
    StoreKey     INT NOT NULL,
    CountryKey   INT NOT NULL,
    ProductKey   INT NOT NULL,
    SaleTimestamp DATETIME2(3) NOT NULL,        -- When the sale occurred (business time)
    Quantity     INT NOT NULL,
    UnitPrice    DECIMAL(18,2) NOT NULL,
    Amount       DECIMAL(18,2) NOT NULL,
    InsertedAt   DATETIME2(3) NOT NULL DEFAULT SYSUTCDATETIME(), -- When record was inserted (system time)
    CONSTRAINT FK_FactSales_Store FOREIGN KEY (StoreKey) REFERENCES dbo.DimStore(StoreKey),
    CONSTRAINT FK_FactSales_Country FOREIGN KEY (CountryKey) REFERENCES dbo.DimCountry(CountryKey),
    CONSTRAINT FK_FactSales_Product FOREIGN KEY (ProductKey) REFERENCES dbo.DimProduct(ProductKey)
  );
  CREATE INDEX IX_FactSales_SaleTimestamp ON dbo.FactSales(SaleTimestamp);
  CREATE INDEX IX_FactSales_InsertedAt ON dbo.FactSales(InsertedAt);
  CREATE INDEX IX_FactSales_ProductKey ON dbo.FactSales(ProductKey);
END;
''')
cursor.execute(ddl)
conn.commit()
print('Schema and tables ensured (Country, Store, Product, Sales with enhanced timestamps).')

Schema and tables ensured (Country, Store, Product, Sales with enhanced timestamps).
